Cargamos el archivo de reviews scrapeadas.

In [14]:
import pandas as pd

flybondi_data = '../data/all_reviews.csv'
df = pd.read_csv(flybondi_data)

## Empezamos a limpiar.

1. Remover columnas inútiles y filas duplicadas.

In [15]:
# remove the unnamed columns which are trash
df_cleaned = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# remove duplicates
df_cleaned = df_cleaned.drop_duplicates()

Limpiamos ratings para que tenga unico formato.

In [16]:
df_cleaned['rating_cleaned'] = df_cleaned['rating'].astype(str).str.extract(r'(\d+\.?\d*)').astype(float)
df_cleaned = df_cleaned.drop(columns=['rating'])
df_cleaned = df_cleaned.rename(columns={'rating_cleaned': 'rating'})
df_cleaned['rating'] = df_cleaned['rating'].fillna(1.0)

df_cleaned.iloc[1300:] 




,name,experience,review_text,likes,review_title,rating
1300,RUBEN OMAR N,NaN,"Es una aerolínea de bajo costo, los servicios ...",NaN,No esperaba nada mas por el precio del Ticket,3.0
1301,Julio F,NaN,Realmente sorprendido por ser una aerolineas L...,NaN,Muy Bueno,4.0
1302,Nelson G,NaN,"Muchas representaciones de vuelos, tanto de id...",NaN,Desastre,1.0
1303,victoriaguida,NaN,Súper económico y el servicio no tiene nada qu...,NaN,Buen servicio,4.0
1304,hector_regina,NaN,"Nadir te avisa nada, teniamos que viajar y el ...",NaN,MUY MAL TRATO,1.0
...,...,...,...,...,...,...
1859,Pablo Romero,1 opinión,Si quieren saber el significado de las palabra...,NaN,SERVICIO NEFASTO FLYBONDI,1.0
1860,Aurélien C,6 opiniones,"Compré un primer boleto, el cheque se cerró co...",NaN,Estafa total,1.0
1861,Marcos Medvescig,1 opinión,La peor experience. Me cambiarion el horario d...,NaN,La peor basura voladora del mundo,1.0
1862,Silvia Elena Perez Sbarbatti,1 opinión,La empresa cumplió con las condiciones pautada...,NaN,"Excelente la puntualidad, la atención y el sev...",5.0


Concatenamos titulos con reviews.

In [17]:
df_cleaned['review'] = df_cleaned['review_title'].fillna('') + '. ' + df_cleaned['review_text'].fillna('')
df_cleaned = df_cleaned.drop(columns=['review_title', 'review_text'])
df_cleaned['review'] = df_cleaned['review'].str.lower()
df_cleaned.tail()


,name,experience,likes,rating,review
1859,Pablo Romero,1 opinión,NaN,1.0,servicio nefasto flybondi. si quieren saber el...
1860,Aurélien C,6 opiniones,NaN,1.0,"estafa total. compré un primer boleto, el cheq..."
1861,Marcos Medvescig,1 opinión,NaN,1.0,la peor basura voladora del mundo. la peor exp...
1862,Silvia Elena Perez Sbarbatti,1 opinión,NaN,5.0,"excelente la puntualidad, la atención y el sev..."
1863,Nazarena Sebastianelli,2 opiniones,NaN,3.0,realmente lo pensaría dos veces antes…. realme...


Limpiamos "Likes"

In [18]:
df_cleaned['likes'] = df_cleaned['likes'].fillna(0)
df_cleaned['likes'] = df_cleaned['likes'].astype(int)

df_cleaned

,name,experience,likes,rating,review
0,Vanesa Fraccarolli,4 reseñas · 7 fotos,0,5.0,. impecable la experiencia con flybondi ida y ...
1,Betiana Tetti,Local Guide · 703 reseñas · 3.139 fotos,3,5.0,. la verdad que siempre que viajé flybondi fue...
2,Valeria Simplituca,22 reseñas · 13 fotos,4,1.0,. una vergüenza y un desastre!!! cambian las c...
3,Aldana Santandreu,6 reseñas · 6 fotos,1,5.0,. tanto el vuelo de ida como de vuelta a baril...
4,Elian Pinel,Local Guide · 104 reseñas · 422 fotos,3,5.0,. viajamos sin ningún tipo de contratiempo a b...
...,...,...,...,...,...
1859,Pablo Romero,1 opinión,0,1.0,servicio nefasto flybondi. si quieren saber el...
1860,Aurélien C,6 opiniones,0,1.0,"estafa total. compré un primer boleto, el cheq..."
1861,Marcos Medvescig,1 opinión,0,1.0,la peor basura voladora del mundo. la peor exp...
1862,Silvia Elena Perez Sbarbatti,1 opinión,0,5.0,"excelente la puntualidad, la atención y el sev..."


Creamos una función que transforma la experience y los likes en un único formato.

In [19]:
import re

def parse_experience(experience):
    resenas = 0
    fotos = 0
    local_guide = 0

    if pd.isna(experience):
        return resenas, fotos, local_guide

    if 'Local Guide' in experience:
        local_guide = 1

    resenas_match = re.search(r'(\d+[\.,]?\d*) (reseñas|opinión|opiniones)', experience)
    if resenas_match:
        resenas = int(resenas_match.group(1).replace('.', '').replace(',', '.'))

    fotos_match = re.search(r'(\d+[\.,]?\d*) fotos', experience)
    if fotos_match:
        fotos = int(fotos_match.group(1).replace('.', '').replace(',', '.'))

    return resenas, fotos, local_guide

df_cleaned[['given_reviews', 'pictures', 'local_guide']] = df_cleaned['experience'].apply(
    lambda x: pd.Series(parse_experience(x))
)

print(df_cleaned[['given_reviews', 'pictures', 'local_guide']])
df_cleaned = df_cleaned.drop(columns=['experience'])


      given_reviews  pictures  local_guide
0                 4         7            0
1               703      3139            1
2                22        13            0
3                 6         6            0
4               104       422            1
...             ...       ...          ...
1859              1         0            0
1860              6         0            0
1861              1         0            0
1862              1         0            0
1863              2         0            0

[1863 rows x 3 columns]


In [20]:
def calculate_relevance(row, W_l=0.3, W_r=0.5, W_p=0.005, W_lg=0.5):
    relevance = (
        W_l * row['likes'] +       
        W_r * row['given_reviews'] +     
        W_p * row['pictures'] +       
        W_lg * row['local_guide']  
    )
    return relevance

df_cleaned['relevance_score'] = df_cleaned.apply(calculate_relevance, axis=1)
df_cleaned['relevance_score_normalized'] = (df_cleaned['relevance_score'] - df_cleaned['relevance_score'].min()) / (df_cleaned['relevance_score'].max() - df_cleaned['relevance_score'].min())

print(df_cleaned[['likes', 'given_reviews', 'pictures', 'local_guide', 'relevance_score', 'relevance_score_normalized']])

      likes  given_reviews  pictures  local_guide  relevance_score  \
0         0              4         7            0            2.035   
1         3            703      3139            1          368.595   
2         4             22        13            0           12.265   
3         1              6         6            0            3.330   
4         3            104       422            1           55.510   
...     ...            ...       ...          ...              ...   
1859      0              1         0            0            0.500   
1860      0              6         0            0            3.000   
1861      0              1         0            0            0.500   
1862      0              1         0            0            0.500   
1863      0              2         0            0            1.000   

      relevance_score_normalized  
0                       0.003946  
1                       0.714761  
2                       0.023784  
3                  

In [21]:
df_cleaned

,name,likes,rating,review,given_reviews,pictures,local_guide,relevance_score,relevance_score_normalized
0,Vanesa Fraccarolli,0,5.0,. impecable la experiencia con flybondi ida y ...,4,7,0,2.035,0.003946
1,Betiana Tetti,3,5.0,. la verdad que siempre que viajé flybondi fue...,703,3139,1,368.595,0.714761
2,Valeria Simplituca,4,1.0,. una vergüenza y un desastre!!! cambian las c...,22,13,0,12.265,0.023784
3,Aldana Santandreu,1,5.0,. tanto el vuelo de ida como de vuelta a baril...,6,6,0,3.330,0.006457
4,Elian Pinel,3,5.0,. viajamos sin ningún tipo de contratiempo a b...,104,422,1,55.510,0.107642
...,...,...,...,...,...,...,...,...,...
1859,Pablo Romero,0,1.0,servicio nefasto flybondi. si quieren saber el...,1,0,0,0.500,0.000970
1860,Aurélien C,0,1.0,"estafa total. compré un primer boleto, el cheq...",6,0,0,3.000,0.005817
1861,Marcos Medvescig,0,1.0,la peor basura voladora del mundo. la peor exp...,1,0,0,0.500,0.000970
1862,Silvia Elena Perez Sbarbatti,0,5.0,"excelente la puntualidad, la atención y el sev...",1,0,0,0.500,0.000970


In [22]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)  # Returns a language code (e.g., 'en', 'es')
    except LangDetectException:
        return 'unknown'  # Handle cases where language detection fails

df_cleaned['language'] = df_cleaned['review'].apply(detect_language)

# Display the DataFrame with the new 'language' column
print(df_cleaned[['review', 'language']])


                                                 review language
0     . impecable la experiencia con flybondi ida y ...       es
1     . la verdad que siempre que viajé flybondi fue...       es
2     . una vergüenza y un desastre!!! cambian las c...       es
3     . tanto el vuelo de ida como de vuelta a baril...       es
4     . viajamos sin ningún tipo de contratiempo a b...       es
...                                                 ...      ...
1859  servicio nefasto flybondi. si quieren saber el...       es
1860  estafa total. compré un primer boleto, el cheq...       es
1861  la peor basura voladora del mundo. la peor exp...       es
1862  excelente la puntualidad, la atención y el sev...       es
1863  realmente lo pensaría dos veces antes…. realme...       es

[1863 rows x 2 columns]


In [23]:
from googletrans import Translator
from googletrans import LANGUAGES

translator = Translator()

def translate_to_spanish(text, src_lang):
    try:
        translation = translator.translate(text, src=src_lang, dest='es')  # 'es' for Spanish
        return translation.text
    except Exception as e:
        print("could not translate: ", text)
        print(f"Error translating: {e}")
        return text
    
def translate_non_spanish(text, lang):
    if lang != 'es' and lang != 'unknown':
        return translate_to_spanish(text, lang)
    return text 

df_cleaned['review_translated'] = df_cleaned.apply(
    lambda row: translate_non_spanish(row['review'], row['language']),
    axis=1
)

print(df_cleaned[['review', 'language', 'review_translated']])

# Optionally, save the updated DataFrame
output_file_translated_reviews = 'cleaned_with_translated_non_es_reviews.csv'
df_cleaned.to_csv(output_file_translated_reviews, index=False)


Error translating: 'Translator' object has no attribute 'raise_Exception'
Error translating: 'Translator' object has no attribute 'raise_Exception'
Error translating: 'Translator' object has no attribute 'raise_Exception'
Error translating: 'Translator' object has no attribute 'raise_Exception'
Error translating: 'Translator' object has no attribute 'raise_Exception'
Error translating: 'Translator' object has no attribute 'raise_Exception'
                                                 review language  \
0     . impecable la experiencia con flybondi ida y ...       es   
1     . la verdad que siempre que viajé flybondi fue...       es   
2     . una vergüenza y un desastre!!! cambian las c...       es   
3     . tanto el vuelo de ida como de vuelta a baril...       es   
4     . viajamos sin ningún tipo de contratiempo a b...       es   
...                                                 ...      ...   
1859  servicio nefasto flybondi. si quieren saber el...       es   
1860  estafa

# Lemmatizador
Ahora que tenemos un dataset con todas las reviews limpias y traducidas procedemos a lemmatizar el texto.

In [3]:
import stanza
import pandas as pd

flybondi_data = '../data/cleaned_with_translated_non_es_reviews.csv'
df = pd.read_csv(flybondi_data)

stanza.download('es') 
nlp = stanza.Pipeline('es')

def lemmatize_spanish(text):
    doc = nlp(text)  
    
    return ' '.join([word.lemma for sent in doc.sentences for word in sent.words])

# Apply the lemmatization function only to Spanish reviews
df['review_lemmatized'] = df.apply(
    lambda row: lemmatize_spanish(row['review_translated']),
    axis=1
)

# Display the DataFrame with the original, translated, and lemmatized reviews
print(df[['review_translated', 'language', 'review_lemmatized']])

# Optionally, save the updated DataFrame with lemmatized reviews
output_file_lemmatized_reviews = 'cleaned_with_lemmatized_reviews.csv'
df.to_csv(output_file_lemmatized_reviews, index=False)


2024-10-18 13:15:04 INFO: Downloaded file to /home/chris/stanza_resources/resources.json
2024-10-18 13:15:04 INFO: Downloading default packages for language: es (Spanish) ...
2024-10-18 13:15:05 INFO: File exists: /home/chris/stanza_resources/es/default.zip
2024-10-18 13:15:21 INFO: Finished downloading models and saved to /home/chris/stanza_resources
2024-10-18 13:15:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2024-10-18 13:15:21 INFO: Downloaded file to /home/chris/stanza_resources/resources.json
2024-10-18 13:15:23 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| deppars

KeyboardInterrupt: 